# LangChain

LangChain is the most popular open source library to interact with multiple LLMs and RAG systems in a unified framework. You may love it, you may hate it, but you have to know how to use it!

Let us try to make a system similar to the previous notebook in a few easy steps. The first thing to do:

`pip install -qU "langchain[google-genai]"`

⚠️LangChain changes all the time. This notebook was generated using the 0.3.27 version.

How to call the model using LangChain linguo:

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash-lite", model_provider="google_genai")

model.invoke("I like frogs")

As you can see, it is easy to call the model from here. The output also has many different and handy metadata that comes with it. Langchain even goes to look for the API key on its own in the files around. As long as you call it GOOGLE_API_KEY, it'll find it.

Let's check what the "model" variable is.

In [ ]:
print(type(model))

As you can see, langchain has a special object for the google generative models. In fact it does for every major provider, and you can interact with all of them using the same langchain functions, this is one of the reasons it makes for such a useful package.

For example if you want to simulate a chat with a model, you may do so in lanchain using LLMChain.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

# 1 – prompt that leaves a slot for prior turns
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),    # <── chat lives here
    ("human", "{input}")
])

# 2 – memory that feeds the placeholder
memory = ConversationBufferMemory(memory_key="chat_history",
                                  return_messages=True)
chat  = LLMChain(llm=model, prompt=prompt, memory=memory)

# 3 – minimal interactive loop
while True:
    user = input("You: ")
    if user.lower() in {"exit", "quit"}:
        break
    print("You:", user)
    print("AI:", chat.predict(input=user))

It is also much easier to perform RAG using langchain. Take the example from our previous notebook. Using langchain to build this rag system is a piece of cake. It may look like some alien language at first, especially with all the objects and classes. But rest assured that langchain allows for building in all directions.

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# --- 1) Load & chunk your document ---
path = "data\\becode_rules.txt"
docs = TextLoader(path, encoding="utf-8").load()

splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=80)
chunks = splitter.split_documents(docs)

# --- 2) Build a tiny local index (FAISS) with Google embeddings ---
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vs = FAISS.from_documents(chunks, embeddings)
retriever = vs.as_retriever(search_kwargs={"k": 2})

# --- 3) Minimal RAG prompt & chain ---
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You answer using ONLY the context. If unsure, say you don't know.\n\nContext:\n{context}"),
    ("human", "{question}")
])

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {  # fetch context from retriever, pass the raw question through
      "context": retriever | format_docs,
      "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

# --- 4) Ask stuff ---
print(rag_chain.invoke("When are the Moodle check-ins?"))
print(rag_chain.invoke("How long can internships last in Brussels?"))

If you want to dive deeper on LangChain. It is an immensely popular module, there are plenty of online resources about it! You may start on [Datacamp](https://app.datacamp.com/learn/courses/developing-llm-applications-with-langchain). Good luck! And have Fun!